In [ ]:
%matplotlib inline
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy import signal, stats
from sklearn import linear_model
import sys
import warnings; warnings.filterwarnings("ignore")

from aux import get_seg
from disp import set_font_size
from my_stats import nanpearsonr

from record import smlt_ppln_ma, smlt_ppln_lin, smlt_ppln_lin2e, smlt_ppln_ln, smlt_ppln_ln2e, smlt_ppln_masoff, smlt_ppln_masrb

cc = np.concatenate

FPS_DN = 8.5  # sampling rate from neural data
DT_DN = 1/FPS_DN
NT_H_DN = 85
T_H_DN = np.arange(NT_H_DN)*DT_DN

FPS = 30.03  # sampling rate of behavioral data
DT = 1/FPS
NT_H = int(round(NT_H_DN*FPS/FPS_DN))
T_H = np.arange(NT_H)*DT

H_S_COLS = [f'IT_H_S_{it}' for it in range(NT_H_DN)]
H_P_COLS = [f'IT_H_P_{it}' for it in range(NT_H_DN)]

Load either Coen or WT Gold behavior data

In [ ]:
df_behav = pd.read_csv('data/simple/c_song_f_behav.csv')
CTR_OFFSET = 0

# df_behav = pd.read_csv('data/simple/w_song_f_behav.csv')
# CTR_OFFSET = 276

df_behav

In [ ]:
# split big df into dfs for individual trials
ntr = np.max(df_behav.ID) + 1
dfs_tr = [df_behav[df_behav.ID == i] for i in range(ntr)]

tr_lens = np.array([len(df_tr) for df_tr in dfs_tr])
tr_lens_cum = cc([[0], np.cumsum(tr_lens)])

n_t_total = np.sum(tr_lens)
assert n_t_total == tr_lens_cum[-1]
df_behav = None

# MA
Multiplicative-adaptation dynamical system neural responses

In [ ]:
SAVE_DIR_MA = 'data/simple/mlv/neur_basic/baker_ma'
PFX_MA = 'mlv_baker_ma'

if not os.path.exists(SAVE_DIR_MA):
    os.makedirs(SAVE_DIR_MA)
    
# neural response fits from Baker data
df_ma_param = pd.read_csv('data/simple/neur/baker_ma_fit_param.csv')

In [ ]:
expt_ids = df_ma_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_ma_param['TAU_R'])
tau_as = np.array(df_ma_param['TAU_A'])
x_ss = np.array(df_ma_param['X_S'])
x_ps = np.array(df_ma_param['X_P'])

In [ ]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_ma = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    # simulate population response
    rs = smlt_ppln_ma(i_s, i_p, tau_rs, tau_as, x_ss, x_ps, DT)
        
    df_tr_nrl_ma[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
        
    np.save(os.path.join(SAVE_DIR_MA, f'{PFX_MA}_tr_{ctr+CTR_OFFSET}.npy'), np.array([{'df': df_tr_nrl_ma}]))

# LIN
Linear neural responses

In [ ]:
SAVE_DIR_LIN = 'data/simple/mlv/neur_basic/baker_lin'
PFX_LIN = 'mlv_baker_lin'

if not os.path.exists(SAVE_DIR_LIN):
    os.makedirs(SAVE_DIR_LIN)
    
# neural response fits from Baker data
df_lin_param = pd.read_csv('data/simple/neur/baker_lin_fit_param.csv')

In [ ]:
expt_ids = df_lin_param['EXPT_ID']

# extract filters (downsampled)
hs_dn_s = np.array([df_lin_param[expt_ids == expt_id].iloc[0][H_S_COLS] for expt_id in expt_ids]).astype(float)
hs_dn_p = np.array([df_lin_param[expt_ids == expt_id].iloc[0][H_P_COLS] for expt_id in expt_ids]).astype(float)

# upsample filters via interpolation
hs_s = np.array([np.interp(T_H, T_H_DN, cc([[0], h_dn_s])[:-1]) for h_dn_s in hs_dn_s])
hs_p = np.array([np.interp(T_H, T_H_DN, cc([[0], h_dn_p])[:-1]) for h_dn_p in hs_dn_p])

fig, axs = plt.subplots(2, 4, figsize=(12, 5), tight_layout=True)
for cexpt, ax_col in zip([0, 10, 30, 50], axs.T):
    ax_col[0].plot(T_H_DN, hs_dn_s[cexpt], c='b', lw=2)
    ax_col[0].plot(T_H, hs_s[cexpt], c='k', lw=1)
    ax_col[0].set_title(f'EXPT_ID {expt_ids[cexpt]}: Sine')
    
    ax_col[1].plot(T_H_DN, hs_dn_p[cexpt], c='r', lw=2)
    ax_col[1].plot(T_H, hs_p[cexpt], c='k', lw=1)
    ax_col[1].set_title(f'EXPT_ID {expt_ids[cexpt]}: Pulse')
    
set_font_size(axs, 12)

fig, axs = plt.subplots(2, 4, figsize=(12, 5), tight_layout=True)
for cexpt, ax_col in zip([0, 10, 30, 50], axs.T):
    ax_col[0].plot(T_H_DN, hs_dn_s[cexpt].cumsum()*DT_DN, c='b', lw=2)
    ax_col[0].plot(T_H, hs_s[cexpt].cumsum()*DT, c='k', lw=1)
    ax_col[0].set_title(f'EXPT_ID {expt_ids[cexpt]}: Sine (step/cum)')
    
    ax_col[1].plot(T_H_DN, hs_dn_p[cexpt].cumsum()*DT_DN, c='r', lw=2)
    ax_col[1].plot(T_H, hs_p[cexpt].cumsum()*DT, c='k', lw=1)
    ax_col[1].set_title(f'EXPT_ID {expt_ids[cexpt]}: Pulse (step/cum)')
    
set_font_size(axs, 12)

Neural responses are given by convolution of each neuron's impulse response with song

$$r(t) = h_{sine}(t) \circledast I_{sine}(t) + h_{pulse}(t) \circledast I_{pulse}(t) = \int_0^{\infty} h_{sine}(t') I_{sine}(t-t')dt' + \int_0^{\infty} h_{pulse}(t') I_{pulse}(t-t')dt'$$

In [ ]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_lin = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    rs = smlt_ppln_lin(i_s, i_p, hs_s.T, hs_p.T, DT)
        
    df_tr_nrl_lin[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
    
    np.save(os.path.join(SAVE_DIR_LIN, f'{PFX_LIN}_tr_{ctr+CTR_OFFSET}.npy'), np.array([{'df': df_tr_nrl_lin}]))

# LIN-2E
Two-exponential linear neural responses. Each filter is sum of two exponentials.

In [ ]:
SAVE_DIR_LIN2E = 'data/simple/mlv/neur_basic/baker_lin2e'
PFX_LIN2E = 'mlv_baker_lin2e'

if not os.path.exists(SAVE_DIR_LIN2E):
    os.makedirs(SAVE_DIR_LIN2E)
    
# neural response fits from Baker data
df_lin2e_param = pd.read_csv('data/simple/neur/baker_lin2e_fit_param.csv')

In [ ]:
expt_ids = df_lin2e_param['EXPT_ID']

# get parameters
x_s_0s = np.array(df_lin2e_param['X_S_0'])
tau_s_0s = np.array(df_lin2e_param['TAU_S_0'])
x_s_1s = np.array(df_lin2e_param['X_S_1'])
tau_s_1s = np.array(df_lin2e_param['TAU_S_1'])
x_p_0s = np.array(df_lin2e_param['X_P_0'])
tau_p_0s = np.array(df_lin2e_param['TAU_P_0'])
x_p_1s = np.array(df_lin2e_param['X_P_1'])
tau_p_1s = np.array(df_lin2e_param['TAU_P_1'])

In [ ]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_lin2e = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
         
    rs = smlt_ppln_lin2e(i_s, i_p, T_H, x_s_0s, tau_s_0s, x_s_1s, tau_s_1s, x_p_0s, tau_p_0s, x_p_1s, tau_p_1s, DT)
    
    df_tr_nrl_lin2e[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
    
    np.save(os.path.join(SAVE_DIR_LIN2E, f'{PFX_LIN2E}_tr_{ctr+CTR_OFFSET}.npy'), np.array([{'df': df_tr_nrl_lin2e}]))

# LIN-R
Linear neural responses (fit with ridge regression)

In [ ]:
SAVE_DIR_LINR = 'data/simple/mlv/neur_basic/baker_linr'
PFX_LINR = 'mlv_baker_linr'

if not os.path.exists(SAVE_DIR_LINR):
    os.makedirs(SAVE_DIR_LINR)
    
# neural response fits from Baker data
df_linr_param = pd.read_csv('data/simple/neur/baker_linr_fit_param.csv')

In [ ]:
expt_ids = df_linr_param['EXPT_ID']

# extract filters (downsampled)
hs_dn_s = np.array([df_linr_param[expt_ids == expt_id].iloc[0][H_S_COLS] for expt_id in expt_ids]).astype(float)
hs_dn_p = np.array([df_linr_param[expt_ids == expt_id].iloc[0][H_P_COLS] for expt_id in expt_ids]).astype(float)

# upsample filters via interpolation
hs_s = np.array([np.interp(T_H, T_H_DN, cc([[0], h_dn_s])[:-1]) for h_dn_s in hs_dn_s])
hs_p = np.array([np.interp(T_H, T_H_DN, cc([[0], h_dn_p])[:-1]) for h_dn_p in hs_dn_p])

fig, axs = plt.subplots(2, 4, figsize=(12, 5), tight_layout=True)
for cexpt, ax_col in zip([0, 10, 30, 50], axs.T):
    ax_col[0].plot(T_H_DN, hs_dn_s[cexpt], c='b', lw=2)
    ax_col[0].plot(T_H, hs_s[cexpt], c='k', lw=1)
    ax_col[0].set_title(f'EXPT_ID {expt_ids[cexpt]}: Sine')
    
    ax_col[1].plot(T_H_DN, hs_dn_p[cexpt], c='r', lw=2)
    ax_col[1].plot(T_H, hs_p[cexpt], c='k', lw=1)
    ax_col[1].set_title(f'EXPT_ID {expt_ids[cexpt]}: Pulse')
    
set_font_size(axs, 12)

fig, axs = plt.subplots(2, 4, figsize=(12, 5), tight_layout=True)
for cexpt, ax_col in zip([0, 10, 30, 50], axs.T):
    ax_col[0].plot(T_H_DN, hs_dn_s[cexpt].cumsum()*DT_DN, c='b', lw=2)
    ax_col[0].plot(T_H, hs_s[cexpt].cumsum()*DT, c='k', lw=1)
    ax_col[0].set_title(f'EXPT_ID {expt_ids[cexpt]}: Sine (step/cum)')
    
    ax_col[1].plot(T_H_DN, hs_dn_p[cexpt].cumsum()*DT_DN, c='r', lw=2)
    ax_col[1].plot(T_H, hs_p[cexpt].cumsum()*DT, c='k', lw=1)
    ax_col[1].set_title(f'EXPT_ID {expt_ids[cexpt]}: Pulse (step/cum)')
    
set_font_size(axs, 12)

In [ ]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_linr = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    rs = smlt_ppln_lin(i_s, i_p, hs_s.T, hs_p.T, DT)
        
    df_tr_nrl_linr[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
    
    np.save(os.path.join(SAVE_DIR_LINR, f'{PFX_LINR}_tr_{ctr+CTR_OFFSET}.npy'), np.array([{'df': df_tr_nrl_linr}]))

# LN
Linear-nonlinear neural responses

In [ ]:
SAVE_DIR_LN = 'data/simple/mlv/neur_basic/baker_ln'
PFX_LN = 'mlv_baker_ln'

if not os.path.exists(SAVE_DIR_LN):
    os.makedirs(SAVE_DIR_LN)
    
# neural response fits from Baker data
df_ln_param = pd.read_csv('data/simple/neur/baker_ln_fit_param.csv')

In [ ]:
expt_ids = df_ln_param['EXPT_ID']

r_mins = np.array([df_ln_param[expt_ids == expt_id].iloc[0]['R_MIN'] for expt_id in expt_ids])
r_maxs = np.array([df_ln_param[expt_ids == expt_id].iloc[0]['R_MAX'] for expt_id in expt_ids])
z_0s = np.array([df_ln_param[expt_ids == expt_id].iloc[0]['Z_0'] for expt_id in expt_ids])
betas = np.array([df_ln_param[expt_ids == expt_id].iloc[0]['BETA'] for expt_id in expt_ids])

# extract filters (downsampled)
hs_dn_s = np.array([df_ln_param[expt_ids == expt_id].iloc[0][H_S_COLS] for expt_id in expt_ids]).astype(float)
hs_dn_p = np.array([df_ln_param[expt_ids == expt_id].iloc[0][H_P_COLS] for expt_id in expt_ids]).astype(float)

# # upsample filters via interpolation
hs_s = np.array([np.interp(T_H, T_H_DN, cc([[0], h_dn_s])[:-1]) for h_dn_s in hs_dn_s])
hs_p = np.array([np.interp(T_H, T_H_DN, cc([[0], h_dn_p])[:-1]) for h_dn_p in hs_dn_p])

fig, axs = plt.subplots(2, 4, figsize=(12, 5), tight_layout=True)
for cexpt, ax_col in zip([0, 10, 30, 50], axs.T):
    ax_col[0].plot(T_H_DN, hs_dn_s[cexpt], c='b', lw=2)
    ax_col[0].plot(T_H, hs_s[cexpt], c='k', lw=1)
    ax_col[0].set_title(f'EXPT_ID {expt_ids[cexpt]}: Sine')
    
    ax_col[1].plot(T_H_DN, hs_dn_p[cexpt], c='r', lw=2)
    ax_col[1].plot(T_H, hs_p[cexpt], c='k', lw=1)
    ax_col[1].set_title(f'EXPT_ID {expt_ids[cexpt]}: Pulse')
    
set_font_size(axs, 12)

fig, axs = plt.subplots(2, 4, figsize=(12, 5), tight_layout=True)
for cexpt, ax_col in zip([0, 10, 30, 50], axs.T):
    ax_col[0].plot(T_H_DN, hs_dn_s[cexpt].cumsum()*DT_DN, c='b', lw=2)
    ax_col[0].plot(T_H, hs_s[cexpt].cumsum()*DT, c='k', lw=1)
    ax_col[0].set_title(f'EXPT_ID {expt_ids[cexpt]}: Sine (step/cum)')
    
    ax_col[1].plot(T_H_DN, hs_dn_p[cexpt].cumsum()*DT_DN, c='r', lw=2)
    ax_col[1].plot(T_H, hs_p[cexpt].cumsum()*DT, c='k', lw=1)
    ax_col[1].set_title(f'EXPT_ID {expt_ids[cexpt]}: Pulse (step/cum)')
    
set_font_size(axs, 12)

Neural responses are given by convolution of each neuron's impulse response with song, followed by sigmoid nonlin

$$z(t) = h_{sine}(t) \circledast I_{sine}(t) + h_{pulse}(t) \circledast I_{pulse}(t)$$

$$r(t) = r_{min} + (r_{max} - r_{min})\left[\frac{\tanh(\beta(z - z_0)) - 1}{2}\right]$$

In [ ]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_ln = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    rs = smlt_ppln_ln(i_s, i_p, hs_s.T, hs_p.T, r_mins, r_maxs, z_0s, betas, DT)
    df_tr_nrl_ln[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
        
    np.save(os.path.join(SAVE_DIR_LN, f'{PFX_LN}_tr_{ctr+CTR_OFFSET}.npy'), np.array([{'df': df_tr_nrl_ln}]))

# LN-2E
Two-exponential linear-nonlinear neural responses. Each filter is sum of two exponentials.

In [ ]:
SAVE_DIR_LN2E = 'data/simple/mlv/neur_basic/baker_ln2e'
PFX_LN2E = 'mlv_baker_ln2e'

if not os.path.exists(SAVE_DIR_LN2E):
    os.makedirs(SAVE_DIR_LN2E)
    
# neural response fits from Baker data
df_ln2e_param = pd.read_csv('data/simple/neur/baker_ln2e_fit_param.csv')

In [ ]:
expt_ids = df_ln2e_param['EXPT_ID']

# get parameters
x_s_0s = np.array(df_ln2e_param['X_S_0'])
tau_s_0s = np.array(df_ln2e_param['TAU_S_0'])
x_s_1s = np.array(df_ln2e_param['X_S_1'])
tau_s_1s = np.array(df_ln2e_param['TAU_S_1'])
x_p_0s = np.array(df_ln2e_param['X_P_0'])
tau_p_0s = np.array(df_ln2e_param['TAU_P_0'])
x_p_1s = np.array(df_ln2e_param['X_P_1'])
tau_p_1s = np.array(df_ln2e_param['TAU_P_1'])
r_mins = np.array(df_ln2e_param['R_MIN'])
r_maxs = np.array(df_ln2e_param['R_MAX'])
z_0s = np.array(df_ln2e_param['Z_0'])
betas = np.array(df_ln2e_param['BETA'])

In [ ]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_ln2e = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    rs = smlt_ppln_ln2e(i_s, i_p, T_H, x_s_0s, tau_s_0s, x_s_1s, tau_s_1s, x_p_0s, tau_p_0s, x_p_1s, tau_p_1s, r_mins, r_maxs, z_0s, betas, DT)
    df_tr_nrl_ln2e[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
    
    np.save(os.path.join(SAVE_DIR_LN2E, f'{PFX_LN2E}_tr_{ctr+CTR_OFFSET}.npy'), np.array([{'df': df_tr_nrl_ln2e}]))

# LN-R
Linear-nonlinear neural responses (fit with ridge regression)

In [ ]:
SAVE_DIR_LNR = 'data/simple/mlv/neur_basic/baker_lnr'
PFX_LNR = 'mlv_baker_lnr'

if not os.path.exists(SAVE_DIR_LNR):
    os.makedirs(SAVE_DIR_LNR)
    
# neural response fits from Baker data
df_lnr_param = pd.read_csv('data/simple/neur/baker_lnr_fit_param.csv')

In [ ]:
expt_ids = df_lnr_param['EXPT_ID']

r_mins = np.array([df_lnr_param[expt_ids == expt_id].iloc[0]['R_MIN'] for expt_id in expt_ids])
r_maxs = np.array([df_lnr_param[expt_ids == expt_id].iloc[0]['R_MAX'] for expt_id in expt_ids])
z_0s = np.array([df_lnr_param[expt_ids == expt_id].iloc[0]['Z_0'] for expt_id in expt_ids])
betas = np.array([df_lnr_param[expt_ids == expt_id].iloc[0]['BETA'] for expt_id in expt_ids])

# extract filters (downsampled)
hs_dn_s = np.array([df_lnr_param[expt_ids == expt_id].iloc[0][H_S_COLS] for expt_id in expt_ids]).astype(float)
hs_dn_p = np.array([df_lnr_param[expt_ids == expt_id].iloc[0][H_P_COLS] for expt_id in expt_ids]).astype(float)

# # upsample filters via interpolation
hs_s = np.array([np.interp(T_H, T_H_DN, cc([[0], h_dn_s])[:-1]) for h_dn_s in hs_dn_s])
hs_p = np.array([np.interp(T_H, T_H_DN, cc([[0], h_dn_p])[:-1]) for h_dn_p in hs_dn_p])

fig, axs = plt.subplots(2, 4, figsize=(12, 5), tight_layout=True)
for cexpt, ax_col in zip([0, 10, 30, 50], axs.T):
    ax_col[0].plot(T_H_DN, hs_dn_s[cexpt], c='b', lw=2)
    ax_col[0].plot(T_H, hs_s[cexpt], c='k', lw=1)
    ax_col[0].set_title(f'EXPT_ID {expt_ids[cexpt]}: Sine')
    
    ax_col[1].plot(T_H_DN, hs_dn_p[cexpt], c='r', lw=2)
    ax_col[1].plot(T_H, hs_p[cexpt], c='k', lw=1)
    ax_col[1].set_title(f'EXPT_ID {expt_ids[cexpt]}: Pulse')
    
set_font_size(axs, 12)

fig, axs = plt.subplots(2, 4, figsize=(12, 5), tight_layout=True)
for cexpt, ax_col in zip([0, 10, 30, 50], axs.T):
    ax_col[0].plot(T_H_DN, hs_dn_s[cexpt].cumsum()*DT_DN, c='b', lw=2)
    ax_col[0].plot(T_H, hs_s[cexpt].cumsum()*DT, c='k', lw=1)
    ax_col[0].set_title(f'EXPT_ID {expt_ids[cexpt]}: Sine (step/cum)')
    
    ax_col[1].plot(T_H_DN, hs_dn_p[cexpt].cumsum()*DT_DN, c='r', lw=2)
    ax_col[1].plot(T_H, hs_p[cexpt].cumsum()*DT, c='k', lw=1)
    ax_col[1].set_title(f'EXPT_ID {expt_ids[cexpt]}: Pulse (step/cum)')
    
set_font_size(axs, 12)

In [ ]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_lnr = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    rs = smlt_ppln_ln(i_s, i_p, hs_s.T, hs_p.T, r_mins, r_maxs, z_0s, betas, DT)
    df_tr_nrl_lnr[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
    
    np.save(os.path.join(SAVE_DIR_LNR, f'{PFX_LNR}_tr_{ctr+CTR_OFFSET}.npy'), np.array([{'df': df_tr_nrl_lnr}]))

# MA-S-OFF (Q, P)
Multiplicative-adaptation neurons w/ responses to sine-offsets followed by quiet or pulse

In [ ]:
SAVE_DIR_MASOFF = 'data/simple/mlv/neur_basic/baker_masoff'
PFX_MASOFF = 'mlv_baker_masoff'

if not os.path.exists(SAVE_DIR_MASOFF):
    os.makedirs(SAVE_DIR_MASOFF)
    
# neural response fits from Baker data
df_masoff_param = pd.read_csv('data/simple/neur/baker_masoff_fit_param.csv')

In [ ]:
expt_ids = df_masoff_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_masoff_param['TAU_R'])
tau_as = np.array(df_masoff_param['TAU_A'])
x_s_all = np.array(df_masoff_param['X_S'])
x_p_all = np.array(df_masoff_param['X_P'])
x_qs_all = np.array(df_masoff_param['X_QS'])
x_ps_all = np.array(df_masoff_param['X_PS'])

In [ ]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_masoff = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    rs = smlt_ppln_masoff(i_s, i_p, tau_rs, tau_as, x_s_all, x_p_all, x_qs_all, x_ps_all, DT)
    df_tr_nrl_masoff[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
    
    np.save(os.path.join(SAVE_DIR_MASOFF, f'{PFX_MASOFF}_tr_{ctr+CTR_OFFSET}.npy'), np.array([{'df': df_tr_nrl_masoff}]))

# MA-S-OFF (Q)
Multiplicative-adaptation neurons w/ response to sine-offsets followed by quiet only

In [ ]:
SAVE_DIR_MAQS = 'data/simple/mlv/neur_basic/baker_maqs'
PFX_MAQS = 'mlv_baker_maqs'

if not os.path.exists(SAVE_DIR_MAQS):
    os.makedirs(SAVE_DIR_MAQS)
    
# neural response fits from Baker data
df_masoff_param = pd.read_csv('data/simple/neur/baker_masoff_fit_param.csv')

In [ ]:
expt_ids = df_masoff_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_masoff_param['TAU_R'])
tau_as = np.array(df_masoff_param['TAU_A'])
x_s_all = np.array(df_masoff_param['X_S'])
x_p_all = np.array(df_masoff_param['X_P'])
x_qs_all = np.array(df_masoff_param['X_QS'])
x_ps_all = np.array(df_masoff_param['X_PS'])*0

In [ ]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_maqs = df_tr.copy()
    sys.stdout.write('.')
     
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    rs = smlt_ppln_masoff(i_s, i_p, tau_rs, tau_as, x_s_all, x_p_all, x_qs_all, x_ps_all, DT)
    df_tr_nrl_maqs[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
    
    np.save(os.path.join(SAVE_DIR_MAQS, f'{PFX_MAQS}_tr_{ctr+CTR_OFFSET}.npy'), np.array([{'df': df_tr_nrl_maqs}]))

# MA-S-RB (Q, P)
Multiplicative-adaptation neurons w/ "rebound" responses to sine-offset followed by quiet or pulse

$$\tau_r \frac{dr}{dt} = -r + (1-a_s)x_sI_s(t) + (1-a_p)x_pI_p(t) + a_sx_{q/s}I_{q/s} + a_sx_{p/s}I_{p/s}$$

$$\tau_a \frac{da_s}{dt} = -a_s + I_s(t) \quad\quad\quad \tau_a \frac{da_p}{dt} = -a_p + I_p(t)$$

In [ ]:
SAVE_DIR_MAQPSRB = 'data/simple/mlv/neur_basic/baker_maqpsrb'
PFX_MAQPSRB = 'mlv_baker_maqpsrb'

if not os.path.exists(SAVE_DIR_MAQPSRB):
    os.makedirs(SAVE_DIR_MAQPSRB)
    
# neural response fits from Baker data
df_masoff_param = pd.read_csv('data/simple/neur/baker_masoff_fit_param.csv')

In [ ]:
expt_ids = df_masoff_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_masoff_param['TAU_R'])
tau_as = np.array(df_masoff_param['TAU_A'])
x_s_all = np.array(df_masoff_param['X_S'])
x_p_all = np.array(df_masoff_param['X_P'])
x_qs_all = np.array(df_masoff_param['X_QS'])
x_ps_all = np.array(df_masoff_param['X_PS'])

In [ ]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_maqpsrb = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    rs = smlt_ppln_masrb(i_s, i_p, tau_rs, tau_as, x_s_all, x_p_all, x_qs_all, x_ps_all, DT)
    df_tr_nrl_maqpsrb[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
        
    np.save(os.path.join(SAVE_DIR_MAQPSRB, f'{PFX_MAQPSRB}_tr_{ctr+CTR_OFFSET}.npy'), np.array([{'df': df_tr_nrl_maqpsrb}]))

# MA-S-RB (Q)
Multiplicative-adaptation neurons w/ "rebound" responses to sine-offset followed by quiet only

$$\tau_r \frac{dr}{dt} = -r + (1-a_s)x_sI_s(t) + (1-a_p)x_pI_p(t) + a_sx_{q/s}I_{q/s}$$

$$\tau_a \frac{da_s}{dt} = -a_s + I_s(t) \quad\quad\quad \tau_a \frac{da_p}{dt} = -a_p + I_p(t)$$

In [ ]:
SAVE_DIR_MAQSRB = 'data/simple/mlv/neur_basic/baker_maqsrb'
PFX_MAQSRB = 'mlv_baker_maqsrb'

if not os.path.exists(SAVE_DIR_MAQSRB):
    os.makedirs(SAVE_DIR_MAQSRB)
    
# neural response fits from Baker data
df_masoff_param = pd.read_csv('data/simple/neur/baker_masoff_fit_param.csv')

In [ ]:
expt_ids = df_masoff_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_masoff_param['TAU_R'])
tau_as = np.array(df_masoff_param['TAU_A'])
x_s_all = np.array(df_masoff_param['X_S'])
x_p_all = np.array(df_masoff_param['X_P'])
x_qs_all = np.array(df_masoff_param['X_QS'])
x_ps_all = np.array(df_masoff_param['X_PS'])*0

In [ ]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_maqsrb = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    rs = smlt_ppln_masrb(i_s, i_p, tau_rs, tau_as, x_s_all, x_p_all, x_qs_all, x_ps_all, DT)
    df_tr_nrl_maqsrb[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
    
    np.save(os.path.join(SAVE_DIR_MAQSRB, f'{PFX_MAQSRB}_tr_{ctr+CTR_OFFSET}.npy'), np.array([{'df': df_tr_nrl_maqsrb}]))